# Validate business ideas
Some simple methods for validating business ideas with relevant personas. The review is structured as an EDSL survey where the questions are parameterized with the business ideas. Agent personas are created for specified traits and any special instructions.

Inspired by https://twitter.com/mattshumer_/status/1765822278351143113?s=61

In [1]:
# ! pip install edsl

In [2]:
from edsl.questions import QuestionFreeText, QuestionList
from edsl import Scenario, Survey, Agent

## Create agent personas relevant to the review

In [3]:
# Select the number of agent personas to create
num_agents = 5

# List the traits that you want to include in agent personas
agent_traits = ["age", "occupation"]

# Optionally add any special instructions for the agents
special_instructions = """You are a pragmatic business strategist with expertise in dissecting
                     business ideas for real-world applicability."""

# Optionally specify the business type
business_type = "event planning"

# Specify the number of agents that you want to create with the traits and instructions 
def create_agents(num_agents, agent_traits, special_instructions = "", business_type = ""):
    agents = []
    agent_traits = ", ".join(agent_traits)
    
    # Create a list of dicts of traits of that we will use to create the agents
    q = QuestionList(
        question_name = "personas",
        question_text = f"""Draft { num_agents } personas for someone evaluating a business idea for a 
        { business_type } business. Format each persona as a dict where the keys are: persona, { agent_traits }."""
    )

    print("Creating agents:")
    personas = q.run(progress_bar=True).select("personas").to_list()[0]

    # Add any special instructions to each personsa
    [p.update({'special_instructions': special_instructions}) for p in personas]

    # Create the agents
    agents = [Agent(traits = p) for p in personas]
    return agents

agents = create_agents(num_agents, agent_traits, special_instructions, business_type)
agents

Creating agents:


Output()

[Agent(traits = {'persona': 'Entrepreneurial Enthusiast', 'age': 30, 'occupation': 'Business Consultant', 'special_instructions': 'You are a pragmatic business strategist with expertise in dissecting\n                     business ideas for real-world applicability.'}),
 Agent(traits = {'persona': 'Creative Visionary', 'age': 25, 'occupation': 'Freelance Graphic Designer', 'special_instructions': 'You are a pragmatic business strategist with expertise in dissecting\n                     business ideas for real-world applicability.'}),
 Agent(traits = {'persona': 'Corporate Planner', 'age': 40, 'occupation': 'Corporate Events Manager', 'special_instructions': 'You are a pragmatic business strategist with expertise in dissecting\n                     business ideas for real-world applicability.'}),
 Agent(traits = {'persona': 'Social Connector', 'age': 35, 'occupation': 'Public Relations Specialist', 'special_instructions': 'You are a pragmatic business strategist with expertise in disse

## Create some example business ideas
The question text can easily be modified to include more instructions on the examples to generate.

In [4]:
# Specify the number of ideas to generate
num_ideas = 3

# Draft some example business ideas
def draft_ideas(num_ideas, business_type):
    q = QuestionList(
        question_name = "ideas",
        question_text = f"""Draft { num_ideas } original business ideas for a { business_type } business."""
    )
    print("Drafting business ideas:")
    business_ideas = q.run(progress_bar=True).select("ideas").to_list()[0]
    return business_ideas

business_ideas = draft_ideas(num_ideas, business_type)
business_ideas

Drafting business ideas:


Output()

['Eco-Friendly Event Planning',
 'Virtual Reality Experiences',
 'Adventure-Based Team Building']

## Construct the review as a survey
The business ideas are parameters of questions prompting the agents to review them. We can structure the output using different EDSL question types (details on all question types <a href="https://deepnote.com/workspace/expected-parrot-c2fa2435-01e3-451d-ba12-9c36b3b87ad9/project/Expected-Parrot-examples-b457490b-fc5d-45e1-82a5-a66e1738a4b9/notebook/Docs%20-%20Questions-17430978a5634fc4ada3127b6b9bcd66">here</a>).

In [5]:
# Describe the review
review_instructions = """Analyze the business idea objectively, considering its genuine merits and 
                      potential pitfalls, and offer realistic feedback on the idea's utility or lack 
                      thereof. Provide a blunt, balanced validation and recommendation."""

# Specify components to include in each review
review_components = ["business idea overview", "potential markets", "pain points", "market risks", 
                     "alternative business models", "final validation and recommentation"]

# Specify the ideas to review or use the above method to generate some examples
business_ideas = draft_ideas(3, business_type) # ["Idea 1", "Idea 2", "Idea 3"]

# Prompt the agents to review the ideas
def conduct_review(business_type, business_ideas, review_instructions, review_components, agents):
    q1 = QuestionFreeText(
        question_name = "free_text_review",
        question_text = """Consider the following business idea for a """ + business_type + """ 
        business: {{ business_idea }}. """ + review_instructions + """ 
        Include the following components in your review: """ + ", ".join(review_components)
    )
    q2 = QuestionList(
        question_name = "formatted_review",
        question_text = """Consider the following business idea for a """ + business_type + """ 
        business: {{ business_idea }}. """ + review_instructions + """ 
        Provide a review of this idea in the form of a dict with 
        Include the following components in your review, formatting your review as a list where 
        the items are your analyses of the components: """ + ", ".join(review_components)
    )
    survey = Survey([q1, q2])

    # If you want to connect the responses by including the response to q1 in the context for q2
    survey.add_targeted_memory(q2, q1)
    
    scenarios = [Scenario({"business_idea":i}) for i in business_ideas]
    print("Conducting review:")
    results = survey.by(scenarios).by(agents).run(progress_bar=True)
    return results

# review = conduct_review(business_type, business_ideas, review_instructions, review_components, agents)
# review

Drafting business ideas:


Output()

## Putting it all together and inspecting the results

In [6]:
# Create agents with relevant personas and any special instructions
agents = create_agents(num_agents, agent_traits, special_instructions, business_type)

# Creating some example business ideas (replace with actual ideas!)
business_ideas = draft_ideas(num_ideas, business_type)

# Conduct the review as a survey about the ideas
results = conduct_review(business_type, business_ideas, review_instructions, review_components, agents)

Creating agents:


Output()

Drafting business ideas:


Output()

Conducting review:


Output()

In [ ]:
# Inspect the results
results.columns # to show all the components of Results objects

['agent.age',
 'agent.agent_name',
 'agent.occupation',
 'agent.persona',
 'agent.special_instructions',
 'answer.formatted_review',
 'answer.formatted_review_comment',
 'answer.free_text_review',
 'model.frequency_penalty',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_p',
 'model.use_cache',
 'prompt.formatted_review_system_prompt',
 'prompt.formatted_review_user_prompt',
 'prompt.free_text_review_system_prompt',
 'prompt.free_text_review_user_prompt',
 'raw_model_response.formatted_review_raw_model_response',
 'raw_model_response.free_text_review_raw_model_response',
 'scenario.business_idea']

In [ ]:
# Printing select components
results.select("persona", "scenario.*", "free_text_review", "formatted_review").print()

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ agent                ┃ scenario                    ┃ answer                       ┃ answer                      ┃
┃ .persona             ┃ .business_idea              ┃ .free_text_review            ┃ .formatted_review           ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Busy Bee             │ Virtual event planning for  │ As a Corporate Event Manager │ ['Business idea overview:   │
│                      │ remote teams                │ with experience in the       │ Promising due to the        │
│                      │                             │ industry, I can provide an   │ growing trend of remote     │
│                      │                             │ objective analysis of the    │ work and virtual            │
│                      │                             │ business idea for virtual    │ collaboration', 'Potential  │
│                      │                             │ event planning for remote    │ markets: Remote teams,      │
│                      │                             │ teams. The overview of the   │ companies transitioning to  │
│                      │                             │ idea is promising, as there  │ remote work, organizations  │
│                      │                             │ is a growing trend towards   │ seeking virtual             │
│                      │                             │ remote work and virtual      │ team-building solutions',   │
│                      │                             │ collaboration, creating a    │ 'Pain points: Addressing    │
│                      │                             │ potential market for virtual │ the need for engaging and   │
│                      │                             │ event planning services. The │ interactive virtual events  │
│                      │                             │ pain points this idea        │ to enhance team cohesion    │
│                      │                             │ addresses include the need   │ and productivity', 'Market  │
│                      │                             │ for engaging and interactive │ risks: Competition from     │
│                      │                             │ virtual events to foster     │ established event planning  │
│                      │                             │ team cohesion and            │ companies, challenge of     │
│                      │                             │ productivity. However, there │ creating impactful virtual  │
│                      │                             │ are market risks such as     │ experiences', 'Alternative  │
│                      │                             │ competition from established │ business models:            │
│                      │                             │ event planning companies and │ Specialized virtual         │
│                      │                             │ the challenge of creating    │ team-building activities,   │
│                      │                             │ impactful virtual            │ consulting services for     │
│                      │                             │ experiences. An alternative  │ remote team engagement',    │
│                      │                             │ business model could involve │ 'Final validation and       │
│                      │                             │ offering specialized virtual │ recommendation: Idea has    │
│                      │                             │ team-building activities or  │ merit but requires strong   │
│                      │                             │ consulting services for      │ differentiation strategy    │
│                      │                             │ remote team engagement. In   │ and innovation to stand out │
│                      │                             │ c